In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor

from xgboost import XGBRegressor

import warnings

In [3]:
df = pd.read_csv('data/Train.csv')

In [4]:
df.head()

,pickup_datetime,dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,traffic_congestion_level,pickup_borough,dropoff_borough,tolls_amount,tip_amount,fare_estimate,trip_distance_miles
0,3/22/24 9:10,3/22/24 9:23,40.690045,-74.239699,40.704018,-74.214368,2,high,Other,Other,0.00,0.14,18.40,10.9422
1,8/14/24 10:48,8/14/24 10:52,40.763224,-74.053604,40.759277,-74.066002,1,medium,Other,Other,0.00,1.36,9.99,1.2817
2,6/17/24 3:47,6/17/24 3:55,40.658970,-74.179122,40.628631,-74.180385,1,low,Other,Staten Island,5.67,2.36,20.22,13.8034
3,6/25/24 0:45,6/25/24 1:04,40.696980,-73.773970,40.670444,-73.838194,2,medium,Brooklyn,Brooklyn,0.00,3.80,29.73,24.2266
4,6/10/24 4:10,6/10/24 4:14,40.580638,-73.788101,40.594286,-73.780625,3,low,Brooklyn,Brooklyn,0.00,2.68,11.05,3.4222


In [5]:
X = df.drop(columns=['trip_distance_miles'],axis=1)

In [6]:
X.head()

,pickup_datetime,dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,traffic_congestion_level,pickup_borough,dropoff_borough,tolls_amount,tip_amount,fare_estimate
0,3/22/24 9:10,3/22/24 9:23,40.690045,-74.239699,40.704018,-74.214368,2,high,Other,Other,0.00,0.14,18.40
1,8/14/24 10:48,8/14/24 10:52,40.763224,-74.053604,40.759277,-74.066002,1,medium,Other,Other,0.00,1.36,9.99
2,6/17/24 3:47,6/17/24 3:55,40.658970,-74.179122,40.628631,-74.180385,1,low,Other,Staten Island,5.67,2.36,20.22
3,6/25/24 0:45,6/25/24 1:04,40.696980,-73.773970,40.670444,-73.838194,2,medium,Brooklyn,Brooklyn,0.00,3.80,29.73
4,6/10/24 4:10,6/10/24 4:14,40.580638,-73.788101,40.594286,-73.780625,3,low,Brooklyn,Brooklyn,0.00,2.68,11.05


In [7]:
y = df['trip_distance_miles']

In [8]:
y

0        10.9422
1         1.2817
2        13.8034
3        24.2266
4         3.4222
          ...   
14694     9.4093
14695    12.7578
14696    12.3564
14697    27.0350
14698    11.5151
Name: trip_distance_miles, Length: 14699, dtype: float64

In [9]:
# Create column Transformer with three  types of transformers
num_features = X.select_dtypes(exclude="object").columns
categ_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler 
from sklearn.compose import ColumnTransformer
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,categ_features),
        ("StandardScaler",numeric_transformer,num_features),  
    ]
)

In [10]:
X = preprocessor.fit_transform(X)

In [11]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 191087 stored elements and shape (14699, 29021)>

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [13]:
X_train.shape,X_test.shape

((11759, 29021), (2940, 29021))

In [14]:
#create  and evaluate function to give all metrics after model training
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse= np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae, rmse,r2_square

In [15]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.0032
- Mean Absolute Error: 0.0014
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.6092
- Mean Absolute Error: 3.6401
- R2 Score: 0.6637


Lasso
Model performance for Training set
- Root Mean Squared Error: 4.9150
- Mean Absolute Error: 4.1304
- R2 Score: 0.6117
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.9024
- Mean Absolute Error: 4.1830
- R2 Score: 0.6196


Ridge
Model performance for Training set
- Root Mean Squared Error: 1.4683
- Mean Absolute Error: 1.1542
- R2 Score: 0.9653
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.3000
- Mean Absolute Error: 3.4923
- R2 Score: 0.7073


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 2.6807
- Mean Absolute Error: 2.0616
- R2 Score: 0.8845
-----------------------

In [16]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)



,Model Name,R2_Score
7,CatBoosting Regressor,0.949314
6,XGBRegressor,0.947664
5,Random Forest Regressor,0.944762
4,Decision Tree,0.908652
8,AdaBoost Regressor,0.879821
3,K-Neighbors Regressor,0.820670
2,Ridge,0.707311
0,Linear Regression,0.663708
1,Lasso,0.619554
